In [2]:
import torch
import numpy as np
import torch.nn as nn

/home/swayatta/miniconda3/envs/env1/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
sample = torch.randint(1,100,(1,1,28,28))
sample.shape

torch.Size([1, 1, 28, 28])

In [4]:
sample = torch.randint(1,100,(1,4,4,3))
sample

tensor([[[[29, 41,  2],
          [ 3, 83, 15],
          [44,  6, 86],
          [85, 18, 64]],

         [[31, 44, 56],
          [12,  1, 73],
          [30, 93, 33],
          [65, 54, 97]],

         [[33,  6, 95],
          [56, 16, 22],
          [32, 46, 51],
          [44, 79, 99]],

         [[13, 84, 32],
          [47, 51, 19],
          [ 7, 26, 37],
          [44, 86, 98]]]])

In [5]:
from einops import rearrange, reduce, repeat

In [6]:
#### classes
1. prenorm --> contains layernorm
2. feedforward
3. Attention --> qkv
4. Transformers --> contains depth of multiple layers ?
5. ViT --> Whole architecture, uses Transformers as a block

SyntaxError: invalid syntax (3474692341.py, line 2)

In [ ]:
## Other implementation : https://medium.com/mlearning-ai/vision-transformers-from-scratch-pytorch-a-step-by-step-guide-96c3313c2e0c
1. MyViT class
    a) calls patchify
    b) The patches need to go through a linear projection
        nn. Linear is sufficient? Yes. Flattened input, linearly projected output

: 

In [ ]:
from data import patchify

: 

In [12]:
def patchify(image, height_of_patch, width_of_patch, channel):
    batch,seq_len, height_of_image, width_of_image, channel = image.shape
    assert height_of_image % height_of_patch == 0 and width_of_image % width_of_patch == 0, 'Image dimensions must be divisible by patch dimensions'
    
    num_patches_height = height_of_image // height_of_patch
    num_patches_width  = width_of_image // width_of_patch
    
    return rearrange(image, 'b n (h1 h) (w1 w) c-> b (n h1 w1) h w c', h1= num_patches_height, w1= num_patches_width)

In [14]:

# def patchify(image, height_of_patch, width_of_patch, channel):
#     batch, height_of_image, width_of_image, channel = image.shape
#     assert height_of_image % height_of_patch == 0 and width_of_image % width_of_patch == 0, 'Image dimensions must be divisible by patch dimensions'
    
#     num_patches_height = height_of_image // height_of_patch
#     num_patches_width  = width_of_image // width_of_patch
    
    # return rearrange(image, 'b (h1 h) (w1 w) c-> (b h1 w1) h w c', h1= num_patches_height, w1= num_patches_width)

LINEAR_EMBEDDING_DIM = 8

sample = torch.randint(1,100,(40,1,4,4,3), dtype = torch.float) # B, H, W, C,   The sample needs to be in Float datatype
print(sample.shape)
patches = patchify(sample, 2,2,3)
print(patches.shape)

# so now, we have 40 patches lined up, each is a 2 by 2 image. 
# The paper says to flatten this
patches = torch.flatten(patches, start_dim = 2, end_dim = -1)
print("Patches shape = ",patches.shape) 

# linear mapping
input_dim = patches.shape[-1]
linear_embed_dim = LINEAR_EMBEDDING_DIM
linear_mapper = nn.Linear(input_dim, linear_embed_dim)
linear_maps = linear_mapper(patches)
print(linear_maps.shape)

# adding CLS token
batch_size = linear_maps.shape[0]
cls = nn.Parameter(torch.randn(batch_size,1,linear_embed_dim))
print(cls.shape)

linear_maps_with_cls = torch.cat([cls, linear_maps], dim = 1) # adding CLS token to the beginning of each linearly projected sample in the batch
print(linear_maps_with_cls.shape)

torch.Size([40, 1, 4, 4, 3])
torch.Size([40, 4, 2, 2, 3])
Patches shape =  torch.Size([40, 4, 12])
torch.Size([40, 4, 8])
torch.Size([40, 1, 8])
torch.Size([40, 5, 8])


In [15]:
# Positional Encoding
def getPositionEncoding(seq_len, d, n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return torch.tensor(P, dtype = float)
print( "Positional encodings shape = " ,getPositionEncoding(5,8).shape)
positional_encoding = getPositionEncoding(5,8)

Positional encodings shape =  torch.Size([5, 8])


In [16]:
encodings_to_transformer = (linear_maps_with_cls + positional_encoding).float()
encodings_to_transformer.shape

torch.Size([40, 5, 8])

In [17]:
def project(x):
    num_attention_heads, attention_head_size = 12, 8
    new_x_shape = x.size()[:-1] + (num_attention_heads, attention_head_size)
    return x.view(*new_x_shape)

In [45]:
def forward(x):
    attention_head_size  = 8 # BERT_dim
    num_attention_heads = 12
    input_dim = 8
    query = nn.Linear(8 , num_attention_heads * attention_head_size)
    key = nn.Linear(8 , num_attention_heads * attention_head_size)
    value = nn.Linear(8 , num_attention_heads * attention_head_size)
    
    
    q = query(x)
    k = key(x)
    v = value(x)
    
    q,k,v = project(q), project(k), project(v) # B, N, num_of_attn_heads, head_dim
    softmax = nn.Softmax(dim = -1)
    attention_scores = softmax(torch.matmul(q, k.transpose(-1,-2)))
    context_vector = torch.matmul(attention_scores, v)
    context_vector = rearrange(context_vector, 'b n num_att h -> b n (num_att h)')
    print("cont vec shape = " , context_vector.shape)
    
    dense = nn.Linear(num_attention_heads * attention_head_size, input_dim)
    output = dense(context_vector)
    return output
forward(encodings_to_transformer).shape

cont vec shape =  torch.Size([40, 5, 96])


torch.Size([40, 5, 8])

In [120]:
class MultiHeadAttention(nn.Module):
    def __init__(self, dim, attention_head_size, num_attention_heads) -> None:
        super().__init__()
        total_head_size = num_attention_heads * attention_head_size
        self.query = nn.Linear(dim , total_head_size )
        self.key = nn.Linear(dim , total_head_size)
        self.value = nn.Linear(dim , total_head_size)

        self.softmax = nn.Softmax(dim = -1)
        self.dense = nn.Linear(total_head_size, dim)

    def project(self, x):
        num_attention_heads, attention_head_size = 12, 8
        new_x_shape = x.size()[:-1] + (num_attention_heads, attention_head_size)
        return x.view(*new_x_shape)

    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)
        
        q,k,v = self.project(q), self.project(k), self.project(v) # B, N, num_of_attn_heads, head_dim
        attention_scores = self.softmax(torch.matmul(q, k.transpose(-1,-2)))
        context_vector = torch.matmul(attention_scores, v)
        context_vector = rearrange(context_vector, 'b n num_att h -> b n (num_att h)')
        output = self.dense(context_vector)
        return output            


In [121]:
class Patchify():
    def __init__(self, height_of_patch, width_of_patch) -> None:
        self.h = height_of_patch
        self.w = width_of_patch
    def patchify(self, image):
        batch, seq_len, height_of_image, width_of_image, channel = image.shape
        assert height_of_image % self.h == 0 and width_of_image % self.w == 0, 'Image dimensions must be divisible by patch dimensions'
        
        num_patches_height = height_of_image // self.h
        num_patches_width  = width_of_image // self.w
        
        return rearrange(image, 'b n (h1 h) (w1 w) c-> b (n h1 w1) h w c', h1= num_patches_height, w1= num_patches_width)
                

In [122]:
class Linear_Projection_and_Add_CLS():
    def __init__(self, batch_size, seq_len, height_of_patch, width_of_patch, channel, dim) -> None:
        

        total_patch_dim = height_of_patch * width_of_patch * channel
        self.linear_mapper = nn.Linear(total_patch_dim, dim)
        self.cls = nn.Parameter(torch.randn(batch_size,1,dim))

    def project(self, patches):
        patches = torch.flatten(patches, start_dim = 2, end_dim = -1)
        linear_maps = self.linear_mapper(patches)
        return linear_maps
    
    def add_cls(self, linear_maps):
        # adding CLS token

        linear_maps_with_cls = torch.cat([self.cls, linear_maps], dim = 1) # adding CLS token to the beginning of each linearly projected sample in the batch
        return linear_maps_with_cls

In [130]:
class PositionalEncoding():
    def __init__(self, seq_len, dim) -> None:
        self.seq_len = seq_len
        self.d = dim
        self.n = 10000
    def getPositionEncoding(self):
        P = np.zeros((self.seq_len, self.d))
        for k in range(self.seq_len):
            for i in np.arange(int(self.d/2)):
                denominator = np.power(self.n, 2*i/self.d)
                P[k, 2*i] = np.sin(k/denominator)
                P[k, 2*i+1] = np.cos(k/denominator)
        return torch.tensor(P, dtype = float)


In [123]:
class Transfomer(nn.Module):
    def __init__(self, num_blocks, dim, mlp_dim, attention_head_size, num_attention_heads) -> None:
        super().__init__()
        # assert input_dim == attention_head_size
        self.norm1 = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([MultiHeadAttention(dim, attention_head_size, num_attention_heads) for _ in range(num_blocks)])
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim,dim)
            )

    def forward(self,x):
        for layer in self.layers:
            x = x+layer(self.norm1(x))
            x = x + self.mlp(self.norm2(x))
        return x


In [172]:
class ViT(nn.Module):
    def __init__(self, batch_size, seq_len, height_of_patch, width_of_patch, channel, num_blocks, dim, mlp_dim, attention_head_size, num_attention_heads, num_of_labels) -> None:
        super().__init__()
        self.patchify = Patchify(height_of_patch, width_of_patch) ### TBD
        self.linear_and_cls = Linear_Projection_and_Add_CLS(batch_size, seq_len, height_of_patch, width_of_patch, channel, dim)
        self.pos_enc = PositionalEncoding(seq_len, dim)
        self.transformer = Transfomer(num_blocks, dim, mlp_dim, attention_head_size, num_attention_heads)
        self.linear = nn.Linear(dim, num_of_labels)

    
    
    def forward(self, image):
        patches = self.patchify.patchify(image)
        linear_projections = self.linear_and_cls.project(patches)
        linear_projections_with_cls = self.linear_and_cls.add_cls(linear_projections)
        positional_encoding = self.pos_enc.getPositionEncoding()
        input_to_transformer = (linear_projections_with_cls + positional_encoding).float()
        output_from_transformer = self.transformer(input_to_transformer)
        #use cls here
        print(output.shape)
        output = self.linear(output_from_transformer)
        return output

In [97]:
new_samp = torch.randn(40,1,4,4,3)

In [173]:
batch_size = 40
seq_len = 1
height_of_patch = 2
width_of_patch = 2
channel = 3
num_blocks = 4
dim = 16
mlp_dim = 9
attention_head_size = 8
num_attention_heads = 12
num_of_labels = 2

# it should take the number of labels, channel, seq_len implicitly

vit = ViT(batch_size, seq_len, height_of_patch, width_of_patch, channel, num_blocks, dim, mlp_dim, attention_head_size, num_attention_heads, num_of_labels)
vit(new_samp).shape

RuntimeError: mat1 and mat2 shapes cannot be multiplied (160x4 and 12x16)

In [147]:
import torch
import os
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Compose
import lightning.pytorch as pl
dataset = MNIST(os.getcwd(), download=True, transform=Compose([ToTensor()]))
train_loader = utils.data.DataLoader(dataset, batch_size = 10)

In [148]:
dataiter = iter(train_loader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

torch.Size([10, 1, 28, 28])
torch.Size([10])


In [ ]:
from vit import ViT
vit = ViT(batch_size, height_of_patch, width_of_patch, channel, num_blocks, dim, mlp_dim, attention_head_size, num_attention_heads, num_of_labels)
new_samp = torch.randn(40,1,4,4,1)
print(vit(new_samp).shape)

In [175]:
import yaml
from vit import ViT
import os
import torch
from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
import lightning.pytorch as pl


with open('config/config.yaml', 'rb') as f:
    conf = yaml.safe_load(f.read())    # load the config file


batch_size = conf['batch_size']
height_of_patch = conf['height_of_patch']
width_of_patch = conf['width_of_patch']
channel = conf['channel']
num_blocks = conf['num_blocks']
dim = conf['dim']
mlp_dim = conf['mlp_dim']
attention_head_size = conf['attention_head_size']
num_attention_heads = conf['num_attention_heads']




# Defining lightning module

class LitVit(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.vit = vit

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.vit(x)
        loss = nn.functional.mse_loss(y, x)
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


# Loading MNIST ###

dataset = MNIST(os.getcwd(), download=True, transform=ToTensor())
train_loader = utils.data.DataLoader(dataset)



### Loading a subsample of mnist ###
K = 6000 # enter your length here
subsample_train_indices = torch.randperm(len(dataset))[:K]
train_loader = utils.data.DataLoader(dataset, batch_size=batch_size, sampler=utils.data.SubsetRandomSampler(subsample_train_indices))



num_of_labels = len(train_loader.dataset.classes)


print(num_of_labels)


vit = ViT(batch_size, height_of_patch, width_of_patch, channel, num_blocks, dim, mlp_dim, attention_head_size, num_attention_heads, num_of_labels)
new_samp = torch.randn(40,1,4,4,1)
print(vit(new_samp).shape)



#








10
torch.Size([40, 5, 10])


In [171]:
for x,y in train_loader:
    b,c,img_h,img_w = x.shape
    x = x.view(b,1,img_h, img_w, channel)
    z = vit(x)
    print(z.shape)
    
    loss = nn.functional.cross_entropy(y.float(),z)
    print(loss.item)
    break

torch.Size([40, 197, 10])


RuntimeError: 0D or 1D target tensor expected, multi-target not supported